In [130]:
import vectorbt as vbt
import pandas as pd

In [131]:
start_date = '2009-01-01'
end_date = '2021-10-31'
DOW_30_TICKER = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
alpaca_data = vbt.YFData.download(
     DOW_30_TICKER,
     start=start_date,
     end=end_date,
     interval='1D'
 )


/mnt/recoverData/linuxProgram/workspace/vectorbt_2/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/mnt/recoverData/linuxProgram/workspace/vectorbt_2/__pypackages__/3.9/lib/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Setting missing data points to NaN.
  data = cls.align_index(data, missing=missing_index)


In [132]:
open = alpaca_data.get()[0].stack(dropna=False).rename('open')
high = alpaca_data.get()[1].stack(dropna=False).rename('high')
low = alpaca_data.get()[2].stack(dropna=False).rename('low')
close = alpaca_data.get()[3]
price_close = close.stack(dropna=False).rename('close')

In [133]:
def preprocess_vectorbt(close):
    rsi_30 = vbt.RSI.run(close, 30).rsi.stack(dropna=False).rename(columns = {30 : 'rsi_30'})
    ma_30 = vbt.MA.run(close, 30).ma.stack(dropna=False).rename(columns = {30 : 'ma_30'})
    ma_60 = vbt.MA.run(close, 60).ma.stack(dropna=False).rename(columns = {60 : 'ma_60'})
    res = pd.concat([open,high,low,price_close,rsi_30,ma_30,ma_60],axis=1).reset_index() # is slow because this line
    return res

%timeit preprocess_vectorbt(close)
res = preprocess_vectorbt(close)
res

2.43 s ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,Date,symbol,open,high,low,close,rsi_30,ma_30,ma_60
0,2008-12-31 00:00:00+00:00,AAPL,2.625211,2.679260,2.605973,2.606278,NaN,NaN,NaN
1,2008-12-31 00:00:00+00:00,AMGN,43.437676,44.281939,43.399647,43.924458,NaN,NaN,NaN
2,2008-12-31 00:00:00+00:00,AXP,14.442320,15.069199,14.394099,14.908461,NaN,NaN,NaN
3,2008-12-31 00:00:00+00:00,BA,31.195813,32.290929,31.128306,32.005898,NaN,NaN,NaN
4,2008-12-31 00:00:00+00:00,CAT,29.963724,30.923658,29.963724,30.628820,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
96925,2021-10-29 00:00:00+00:00,UNH,449.991602,456.903744,448.654722,455.992676,65.073663,413.718763,412.824563
96926,2021-10-29 00:00:00+00:00,V,208.105676,212.542125,207.439200,210.652161,44.866255,224.879986,227.021133
96927,2021-10-29 00:00:00+00:00,VZ,51.250939,51.787852,51.163080,51.729282,46.768944,51.809784,52.472365
96928,2021-10-29 00:00:00+00:00,WBA,45.414664,45.821708,45.327440,45.569729,42.857140,46.521764,47.068581


In [134]:

def preprocess_vectorbt_raw(close):
    rsi_30 = vbt.RSI.run(close, 30).rsi.stack(dropna=False).rename(columns = {30 : 'rsi_30'})
    ma_30 = vbt.MA.run(close, 30).ma.stack(dropna=False).rename(columns = {30 : 'ma_30'})
    ma_60 = vbt.MA.run(close, 60).ma.stack(dropna=False).rename(columns = {60 : 'ma_60'})
    res = pd.concat([rsi_30,ma_30,ma_60],axis=1).reset_index() # optimization possible in this line
    return res

%timeit preprocess_vectorbt_raw(close)
res = preprocess_vectorbt_raw(close)
res

68.6 ms ± 574 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


,Date,symbol,rsi_30,ma_30,ma_60
0,2008-12-31 00:00:00+00:00,AAPL,NaN,NaN,NaN
1,2008-12-31 00:00:00+00:00,AMGN,NaN,NaN,NaN
2,2008-12-31 00:00:00+00:00,AXP,NaN,NaN,NaN
3,2008-12-31 00:00:00+00:00,BA,NaN,NaN,NaN
4,2008-12-31 00:00:00+00:00,CAT,NaN,NaN,NaN
...,...,...,...,...,...
96925,2021-10-29 00:00:00+00:00,UNH,65.073663,413.718763,412.824563
96926,2021-10-29 00:00:00+00:00,V,44.866255,224.879986,227.021133
96927,2021-10-29 00:00:00+00:00,VZ,46.768944,51.809784,52.472365
96928,2021-10-29 00:00:00+00:00,WBA,42.857140,46.521764,47.068581
